In [5]:
%pip install paho-mqtt

> NOTE:
Avoid exposing credentials for MQTT broker and OT-2 device like this in real projects.
We include them here purely to make this demo easy to run.

In [ ]:
# @title MQTT Broker and OT-2 Configuration

# NOTE:
# Avoid exposing credentials for MQTT broker and OT-2 device like this in real projects.
# We include them here purely to make this demo easy to run.

host = "248cc294c37642359297f75b7b023374.s2.eu.hivemq.cloud"  # @param {type:"string"}
port = 8883  # @param {type:"integer"}
username = "sgbaird"  # @param {type:"string"}
password = "D.Pq5gYtejYbU#L"  # @param {type:"string"}

# @title OT-2 Configuration
OT2_SERIAL = "OT2CEP20240218R04" # @param {type:"string"}
COMMAND_TOPIC = f"command/ot2/{OT2_SERIAL}/pipette"
STATUS_TOPIC = f"status/ot2/{OT2_SERIAL}/complete"

In [ ]:
import json
import paho.mqtt.client as mqtt
import ssl
import time

# ====== Callback: When connected to the MQTT broker ======
def on_connect(client, userdata, flags, rc):
    print("Orchestrator connected to MQTT broker with result code", rc)
    # Subscribe to OT-2 status updates so we know when the device finishes execution
    client.subscribe(STATUS_TOPIC, qos=2)


# ====== Callback: When a message is received from the subscribed topics ======
def on_message(client, userdata, msg):
    payload = msg.payload.decode("utf-8")
    print(f"Received status update from OT-2 on {msg.topic}: {payload}")


# ====== Create and configure MQTT client ======
client = mqtt.Client()
client.tls_set(tls_version=ssl.PROTOCOL_TLS_CLIENT)
client.username_pw_set(username, password)
client.on_connect = on_connect
client.on_message = on_message
client.connect(host, port)
client.loop_start()


# ====== 1. Send a "mix_color" command to OT-2 ======
mix_payload = {
    "command": {
        "R": 120,  # Volume of Red liquid
        "Y": 50,   # Volume of Yellow liquid
        "B": 80,   # Volume of Blue liquid
        "well": "B2"  # Target well on the plate
    },
    "session_id": "session-001",
    "experiment_id": "exp-001"
}

print(f"Sending mix_color command: {mix_payload}")
client.publish(COMMAND_TOPIC, json.dumps(mix_payload), qos=2)

time.sleep(10)


# ====== 2. Send a "move_sensor_back" command to OT-2 ======
sensor_back_payload = {
    "command": {
        "sensor_status": "done"  # Indicates sensor reading completed
    },
    "session_id": "session-001",
    "experiment_id": "exp-001"
}

print(f"Sending move_sensor_back command: {sensor_back_payload}")
client.publish(COMMAND_TOPIC, json.dumps(sensor_back_payload), qos=2)

time.sleep(10)

client.loop_stop()
client.disconnect()
print("Orchestrator finished and disconnected")

/tmp/ipython-input-6-3866809401.py:31: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Sending mix_color command: {'command': {'R': 120, 'Y': 50, 'B': 80, 'well': 'B2'}, 'session_id': 'session-001', 'experiment_id': 'exp-001'}
Orchestrator connected to MQTT broker with result code 0
Received status update from OT-2 on status/ot2/OT2CEP20240218R04/complete: {"status": {"sensor_status": "in_place"}, "experiment_id": "exp-001", "session_id": "session-001"}
Sending move_sensor_back command: {'command': {'sensor_status': 'done'}, 'session_id': 'session-001', 'experiment_id': 'exp-001'}
Received status update from OT-2 on status/ot2/OT2CEP20240218R04/complete: {"status": {"sensor_status": "charging"}, "experiment_id": "exp-001", "session_id": "session-001"}
Received status update from OT-2 on status/ot2/OT2CEP20240218R04/complete: {"status": {"sensor_status": "charging"}, "experiment_id": "exp-001", "session_id": "session-001"}
Received status update from OT-2 on status/ot2/OT2CEP20240218R04/complete: {"status": {"sensor_status": "charging"}, "experiment_id": "exp-001", "sessi